In [1]:
import cv2 
import os 
import sys 
import numpy as np
import pandas as pd 
import tensorflow as tf 
import matplotlib.pyplot as plt 

In [2]:
from IPython.display import SVG 
from tensorflow.keras.models import Model, Sequential 
from tensorflow.keras.optimizers import Adam, RMSprop, SGD 
from tensorflow.python.keras.utils.vis_utils import plot_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator 
from tensorflow.keras.utils import to_categorical, Sequence, model_to_dot 
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping, LearningRateScheduler 
from tensorflow.keras.layers import Activation, Dense, Input, Flatten, Conv2D, MaxPooling2D, Dropout, BatchNormalization, GlobalAveragePooling2D
from sklearn.model_selection import train_test_split 

In [3]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [4]:
drive_path = "/content/drive/MyDrive/"
source_filename = drive_path + "dataset/cat-and-dog.zip"
extract_folder = "dataset/"

import shutil 
shutil.unpack_archive(source_filename, extract_folder)

In [5]:
IMAGE_SIZE = 100 
BATCH_SIZE = 32 
N_CLASSES = 2 

In [10]:
train_dir = "dataset/training_set/training_set"
test_dir = "dataset/test_set/test_set"

In [15]:
def set_build_model():
    input_tensor = Input(shape = (IMAGE_SIZE, IMAGE_SIZE, 3))

    x = Conv2D(32, kernel_size= (3, 3), padding="same")(input_tensor)
    x = Activation("relu")(x)
    x = MaxPooling2D((2, 2))(x)

    x = Conv2D(64, kernel_size=(3, 3), padding="same")(x)
    x = Activation("relu")(x)
    x = MaxPooling2D((2, 2))(x)

    x = Conv2D(128, kernel_size=(3, 3), padding="same")(x)
    x = Activation("relu")(x)
    x = MaxPooling2D((2, 2))(x)

    x = Flatten()(x)
    x = Dense(64, activation="relu")(x)
    x = Dropout(0.4)(x)
    x = Dense(32, activation="relu")(x)
    output = Dense(1, activation="sigmoid", name="output")(x)

    model = Model(inputs= input_tensor, outputs=output)

    model.summary()

    return model  

In [13]:
train_gen = ImageDataGenerator(
    rescale=1./255, 
    rotation_range=10, 
    width_shift_range=0.1, 
    height_shift_range=0.1, 
    shear_range=0.1, 
    zoom_range=0.1, 
    horizontal_flip= True)

test_gen = ImageDataGenerator(
    rescale=1.0/255)

train_generator = train_gen.flow_from_directory(
    train_dir,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training') 

validation_generator = train_gen.flow_from_directory(
    train_dir, 
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation')

test_generator = test_gen.flow_from_directory(
    test_dir,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary')


Found 8005 images belonging to 2 classes.
Found 0 images belonging to 2 classes.
Found 2023 images belonging to 2 classes.


In [24]:
model = set_build_model()

model.compile(optimizer=Adam(lr=0.001), loss="binary_crossentropy", metrics=["accuracy"])

rlr_cb = ReduceLROnPlateau(monitor="val_loss", factor=0.2, patience=5, mode="min", verbose=1)
ely_cb = EarlyStopping(monitor="val_loss", patience=10, mode="min", verbose=1)

history = model.fit(
    train_generator,
    steps_per_epoch = train_generator.samples // BATCH_SIZE,
    validation_data = validation_generator, 
    validation_steps = validation_generator.samples // BATCH_SIZE,
    epochs = 20,
    callbacks=[rlr_cb, ely_cb])



Model: "model_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 100, 100, 3)]     0         
                                                                 
 conv2d_18 (Conv2D)          (None, 100, 100, 32)      896       
                                                                 
 activation_18 (Activation)  (None, 100, 100, 32)      0         
                                                                 
 max_pooling2d_18 (MaxPoolin  (None, 50, 50, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_19 (Conv2D)          (None, 50, 50, 64)        18496     
                                                                 
 activation_19 (Activation)  (None, 50, 50, 64)        0         
                                                           

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Epoch 1/20
250/250 [==============================] - 39s 152ms/step - loss: 0.6935 - accuracy: 0.5241 - lr: 0.0010
Epoch 2/20
250/250 [==============================] - 37s 148ms/step - loss: 0.6654 - accuracy: 0.5995 - lr: 0.0010
Epoch 3/20
250/250 [==============================] - 37s 148ms/step - loss: 0.6285 - accuracy: 0.6494 - lr: 0.0010
Epoch 4/20
250/250 [==============================] - 37s 148ms/step - loss: 0.5853 - accuracy: 0.6878 - lr: 0.0010
Epoch 5/20
250/250 [==============================] - 37s 148ms/step - loss: 0.5543 - accuracy: 0.7137 - lr: 0.0010
Epoch 6/20
250/250 [==============================] - 37s 147ms/step - loss: 0.5274 - accuracy: 0.7379 - lr: 0.0010
Epoch 7/20
250/250 [==============================] - 38s 150ms/step - loss: 0.4991 - accuracy: 0.7635 - lr: 0.0010
Epoch 8/20
250/250 [==============================] - 37s 147ms/step - loss: 0.4790 - accuracy: 0.7687 - lr: 0.0010
Epoch 9/20
250/250 [==============================] - 37s 148ms/step - l

In [27]:
def get_show_plot_loss_acc(history, epochs):
    #loss, val_loss = history.history["loss"], history.history["val_loss"]
    acc, val_acc = history.history["accuracy"], history.history["accuracy"]

    figure, axes = plt.subplot(1, 1, figsize=(12, 4))
    # axes[0].plot(range(1, epochs+1), loss, label="Training")
    # axes[0].plot(range(1, epochs+1), val_loss, label="Validation")
    # axes[0].legend(loc="best")
    # axes[0].set_title("loss")
    
    axes[1].plot(range(1, epochs+1), acc, label="Training")
    axes[1].plot(range(1, epochs+1), val_acc, label="Validation")
    axes[1].legend(loc="best")
    axes[1].set_title("acc")

    plt.tight_layout()
    plt.show()


In [ ]:
get_show_plot_loss_acc(history, 30)
model.evaluate(flow_test_gen)